In [1]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import time

In [2]:
! pip install urlextract

In [3]:
from urlextract import URLExtract as extract
extractor = extract()

In [4]:
import pandas as pd
import re
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

def extract_id(fileName):
    ogd_id = fileName.partition('_')[0]
    return ogd_id.replace('ogd', '')


def getInformation(url):

  csvList = []
  url = url.replace(" ","%20")
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))

  for elem in x:
    stringElem = str(elem)

    if 'csv' in stringElem:
      fileName = re.search('href="(.*)"', stringElem).group(1)
      csvList.append([fileName, extract_id(fileName)])

  df = pd.DataFrame(csvList, columns= ['fileName', 'ogd_id'])
  print(df)
  #df.to_csv('files/info.csv')

getInformation(url)

                                            fileName ogd_id
0  ogd35_schweizerische_elektrizitaetsbilanz_mona...     35
1                                     ogd39_epec.csv     39
2                     ogd47_vollzugsresultate_pw.csv     47
3                ogd47_vollzugsresultate_pw_2020.csv     47
4                    ogd61_vollzugsresultate_lnf.csv     61
5               ogd61_vollzugsresultate_lnf_2020.csv     61
6           ogd90_potenzialstudien_waermenutzung.csv     90


In [5]:
id = 35
url_dataPackage = 'https://www.uvek-gis.admin.ch/BFE/ogd/' + str(id) + '/datapackage.json'

print(url_dataPackage)

https://www.uvek-gis.admin.ch/BFE/ogd/35/datapackage.json


In [6]:
import requests

x = requests.get(url_dataPackage)
print(x.status_code)

200


In [10]:
# import urllib library
from urllib.request import urlopen

# import json
import json

# store the response of URL
response = urlopen(url_dataPackage)

# storing the JSON response
# from url in data
data_json = json.loads(response.read())

# print the json response
print(data_json)

{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/35/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': 'Année', 'description@de': 'Jahr der Aufnahme', 'description@fr': "Année d'acquisition"}, {'name': 'Monat', 'type': 'integer', 'format': 'default', 'title@de': 'Monat', 'title@fr': 'Mois', 'description@de': 'Monat der Aufnahme', 'description@fr': "Mois d'acquisition", 'constraints': {'minimum': '1', 'maximum': '12'}}, {'name': 'Definitiv', 'type': 'boolean', 'format': 'default', 'title@de': '', 'title@fr': '', 'description@de': '', 'description@fr': '', 'trueValues': ['1'], 'falseValues': ['0']}, {'name': 'Erzeugung_laufwerk_GWh', 'type': 'integer', 'format': 'default', 'title@de': 'Erzeugung Laufwerk GWh', 'title@fr':

In [13]:
id = 35
fileName = 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv'
data_json['resources'][0]['path'] = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/' + str(id) + '/' + fileName

In [ ]:
type(data_json)

dict

In [ ]:
json_data = json.dumps(data_json)

# print the Converted JSON object
print(json_data)

{"profile": "tabular-data-package", "resources": [{"name": "ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv", "path": "https://www.uvek-gis.admin.ch/BFE/ogd/staging35/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv", "profile": "tabular-data-resource", "schema": {"fields": [{"name": "Jahr", "type": "year", "format": "default", "title@de": "Jahr", "title@fr": "Ann\u00e9e", "description@de": "Jahr der Aufnahme", "description@fr": "Ann\u00e9e d'acquisition"}, {"name": "Monat", "type": "integer", "format": "default", "title@de": "Monat", "title@fr": "Mois", "description@de": "Monat der Aufnahme", "description@fr": "Mois d'acquisition", "constraints": {"minimum": "1", "maximum": "12"}}, {"name": "Definitiv", "type": "boolean", "format": "default", "title@de": "", "title@fr": "", "description@de": "", "description@fr": "", "trueValues": ["1"], "falseValues": ["0"]}, {"name": "Erzeugung_laufwerk_GWh", "type": "integer", "format": "default", "title@de": "Erzeugung Laufwerk 

In [14]:
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)